# CV Assignment 2

## Roll no - 20171002

### Requirements :-
##### Libraries:-
* Numpy
* os
* Python 3.x
* scikit-image
* matplotlib
* math
* mpl_toolkits.axes_grid1
* OpenCV 3.4.2.16

### To run this notebook successfully, please ensure the following steps.
* Ensure that all the libraries mentioned above are installed
* Ensure that in the current working directory the folder **Camera_calibration_data** and its entire folder structure exists and is maintained. **This is the input data to notebook.**



### Note:-
* ***If any of the steps are missing/files are missing, then some parts of the code may or may not work***
* *The Results folder contains some outputs saved from the script*
* It is advised to use the testing machine in "plugged in" mode, to avoid core suppression as happens in most modern PCs.
* **Results are uploaded on Microsoft OneDrive at this link, `https://1drv.ms/u/s!AriDLlttrVxZjlrTnQkkLt4AYvUr?e=WSQ4kI`**.Please note that this is hosted on IIITH's Google Drive.
* Resources are also uploaded in case in they are not uploadable on Moodle under the name `CV Assignment 2/images`, at this link **`https://1drv.ms/u/s!AriDLlttrVxZjj1TSuRgGlHwmlgP?e=lRRqWP`**. If need be, copy the directory to the location of this iPython notebook. Please note that this is hosted on IIITH's Microsoft OneDrive.
* ***To run this notebook, download `images` at notebook location and run the notebook***

In [1]:
# Importing libraries
import numpy as np
import os, random
import skimage
from imageio import imread, imwrite
import matplotlib
import matplotlib.pyplot as plt
from math import sqrt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import cv2,math
import time
from imutils import paths
import imutils
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse
%matplotlib inline

In [2]:
# RGB to greyscale
def rgb2gray(image):
    '''
    A function that takes in input an image and returns its greyscale version
    '''
    if (len(image.shape)==2):
        return image
    elif (len(image.shape)==3):
        r = image[:,:,0]
        g = image[:,:,1]
        b = image[:,:,2]
        output = 0.2989 * r + 0.5870 * g + 0.1140 * b
        return output.astype('uint8')

## Task 1

## Image Mosaicing

In [3]:
def applySIFT(image):
    '''
    A function to compute SIFT features.
    '''
    sift = cv2.xfeatures2d.SIFT_create()
    keypointsImage, descriptorImage = sift.detectAndCompute(image,None)
    return keypointsImage, descriptorImage

In [4]:
def getGoodMatches(descriptorImage1, descriptorImage2, factor = 2):
    '''
    A funtion to find dense matches using cv2's Brute Force Matcher. The useful matches are sorted using using a 
    factor of distance.
    '''
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptorImage1, descriptorImage2, k=2)
    print('No. of matches obtained = ',np.asarray(matches).shape)
    goodList = []
    good = []
    for m,n in matches:
        if m.distance < factor*n.distance:
            goodList.append([m])
            good.append(m)
    return goodList, good

In [5]:
def getInliers(mask, num=10):
    '''
    A function to obtain good points for drawing the mask.
    '''
    matchesMask = mask.ravel().tolist()
    indices = []
    for ind in range(len(matchesMask)):
        if matchesMask[ind] == 1:
            indices.append(ind)
    matchesMask = [0]*len(matchesMask)
    np.random.shuffle(indices)
    indices = indices[:num]
    for ind in indices:
            matchesMask[ind] = 1
    return matchesMask

In [6]:
def getExtremePoints(image1CornersPlane2):
    '''
    A fumction to find the corners of an input stitched image.
    '''
    xMin = min(image1CornersPlane2[0][0], image1CornersPlane2[1][0])
    yMin = min(image1CornersPlane2[0][1], image1CornersPlane2[3][1])
    xMax = max(image1CornersPlane2[2][0], image1CornersPlane2[3][0])
    yMax = max(image1CornersPlane2[1][1], image1CornersPlane2[2][1])
    return xMin, yMin, xMax, yMax

In [7]:
def stitch_pano(image1, image2, idx, option = 1):
    '''
    A function to stitch two images. Inbuilt cv2.findHomography is used.
    '''
    keypointsImage1, descriptorImage1 = applySIFT(image1)
    keypointsImage2, descriptorImage2 = applySIFT(image2)

    Image1Keypoints=cv2.drawKeypoints(image1,keypointsImage1,None)
    cv2.imwrite('Results/sift1_'+str(idx)+'.jpg',Image1Keypoints)
    cv2.imshow('keypoints in Image 1',Image1Keypoints)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    Image2Keypoints=cv2.drawKeypoints(image2,keypointsImage2,None)
    cv2.imwrite('Results/sift2_'+str(idx)+'.jpg',Image2Keypoints)
    cv2.imshow('keypoints in Image 2',Image2Keypoints)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

    goodList, good = getGoodMatches(descriptorImage1, descriptorImage2)

    imagePlot = cv2.drawMatchesKnn(image1,keypointsImage1,image2,keypointsImage2,goodList,None,flags=2)
    cv2.imwrite('Results/matches_knn_'+str(idx)+'.jpg',imagePlot)
    cv2.imshow('matches derived by 2nn',imagePlot)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    ptsImage1 = np.array([ keypointsImage1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    ptsImage2 = np.array([ keypointsImage2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

    H, mask = cv2.findHomography(ptsImage1, ptsImage2, cv2.RANSAC)
    print('Homography Matrix:')
    print(H)
 
    matchesMask = getInliers(mask, 10)
    inlierImage = cv2.drawMatches(image1,keypointsImage1,image2,keypointsImage2,good,None,matchesMask = matchesMask,flags = 2)
    cv2.imwrite('Results/matches_'+str(idx)+'.jpg',inlierImage)
    cv2.imshow('matches in image 1',inlierImage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if (option == 1):
        h, w, d = image1.shape
        image1Corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        image1CornersPlane2 = np.squeeze(cv2.perspectiveTransform(image1Corners,H))
        cv2.destroyAllWindows()
    
        xMin, yMin, xMax, yMax = getExtremePoints(image1CornersPlane2)
        t1 = (xMax-xMin, yMax-yMin)
        t2 = (len(image2[0])-int(xMin), len(image2)-int(yMin))
        finalImageShape = max(t1,t2)
        if xMin < 0 and yMin < 0:
            translate = np.float32([[1,0, -xMin], [0,1, -yMin], [0,0,1]])
        elif xMin < 0:
            translate = np.float32([[1,0, -xMin], [0,1,0], [0,0,1]])
        elif xMin < 0:
            translate = np.float32([[1,0,0], [0,1, -yMin], [0,0,1]])
        else:
            translate = np.float32([[1,0,0], [0,1,0], [0,0,1]])
        finalImage = cv2.warpPerspective(image1, np.matmul(translate,H), finalImageShape)
        finalImage[-int(yMin):-int(yMin)+image2.shape[0], -int(xMin):-int(xMin)+image2.shape[1]]=image2
    
        cv2.imwrite('Results/uncrop_pano_'+str(idx)+'.jpg',finalImage)
        cv2.imshow('Uncropped Pano',finalImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
        return finalImage
    
    elif (option == 2):
        h1, w1 = image2.shape[:2]
        h2, w2 = image1.shape[:2]
        c1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
        c2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
        c2_ = cv2.perspectiveTransform(c2, H)
        c = np.concatenate((c1, c2_), axis=0)

        [xmin, ymin] = np.int32(c.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(c.max(axis=0).ravel() + 0.5)
        t = [-xmin, -ymin]

        Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])

        out = cv2.warpPerspective(image1, Ht.dot(H), (xmax-xmin, ymax-ymin))
        out[t[1]:h1+t[1], t[0]:w1+t[0]] = image2
        finalImage = out
        cv2.imwrite('Results/uncrop_pano_'+str(idx)+'.jpg',finalImage)
        cv2.imshow('Uncropped Pano',finalImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return finalImage

In [8]:
def resize(im, scale_percent):
    '''
    A function to resize images for fast computation.
    '''
    width = int(im.shape[1] * scale_percent / 100)
    height = int(im.shape[0] * scale_percent / 100)
    dim = (width, height)
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    return im

In [9]:
scottsdale1=cv2.imread('./images/image_mosaicing/img3_1.png')
scottsdale2=cv2.imread('./images/image_mosaicing/img3_2.png')

In [10]:
stitch_pano(scottsdale1, scottsdale2, 'scottsdale')


No. of matches obtained =  (2240, 2)
Homography Matrix:
[[ 1.92319108e+00 -6.88267538e-02 -1.04475782e+03]
 [ 3.45715051e-01  1.69035404e+00 -2.94587475e+02]
 [ 9.20503413e-04  3.27063438e-05  1.00000000e+00]]


array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [40, 48, 52],
        ...,
        [56, 65, 87],
        [54, 66, 84],
        [52, 66, 84]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [40, 49, 53],
        ...,
        [52, 63, 84],
        [51, 64, 82],
        [51, 65, 83]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [46, 56, 59],
        ...,
        [50, 61, 82],
        [52, 66, 84],
        [54, 69, 86]]], dtype=uint8)

In [11]:
amphi1 = resize(cv2.imread('./images/image_mosaicing/img1_1.jpg'), 40)
amphi2 = resize(cv2.imread('./images/image_mosaicing/img1_2.jpg'), 40)
amphi3 = resize(cv2.imread('./images/image_mosaicing/img1_3.jpg'), 40)
amphi4 = resize(cv2.imread('./images/image_mosaicing/img1_4.jpg'), 40)

In [12]:
stitch_pano(amphi1, amphi2, 'amphi12')
amphi_temp_1 = resize(cv2.imread('./Results/uncrop_pano_amphi12.jpg'), 40)
stitch_pano(amphi_temp_1, amphi3, 'amphi123')

No. of matches obtained =  (9587, 2)
Homography Matrix:
[[ 6.16890885e-01  2.81961166e-01 -9.98020144e+01]
 [-5.52838076e-02  1.01497193e+00  7.10301999e+01]
 [-9.78068275e-05  2.69342591e-04  1.00000000e+00]]
No. of matches obtained =  (1275, 2)
Homography Matrix:
[[ 3.62179765e+00 -3.92186273e-02 -7.65341672e+02]
 [ 2.66623075e-01  3.18800185e+00 -1.78543408e+02]
 [ 5.97486549e-04 -2.47070773e-05  1.00000000e+00]]


array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [41, 50, 54],
        [40, 49, 53],
        [39, 48, 51]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [40, 49, 52],
        [39, 48, 52],
        [41, 50, 53]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [41, 50, 53],
        [40, 49, 52],
        [33, 42, 45]]], dtype=uint8)

In [15]:
amphi_temp_2 = resize(cv2.imread('./Results/uncrop_pano_amphi123.jpg'), 40)
stitch_pano(amphi_temp_2, amphi4, 'amphi')

No. of matches obtained =  (2189, 2)
Homography Matrix:
[[ 3.12337235e+00 -3.18927270e-02 -1.33055870e+03]
 [ 1.73684197e-01  2.85082939e+00 -3.26523634e+02]
 [ 3.90178218e-04  3.79041495e-06  1.00000000e+00]]


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [137, 143, 151],
        [136, 143, 151],
        [133, 139, 147]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [135, 142, 151],
        [135, 142, 151],
        [133, 140, 149]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [136, 144, 154],
        [137, 145, 155],
        [132, 141, 151]]

In [13]:
him1 = resize(cv2.imread('./images/image_mosaicing/img5_1.jpg'), 40)
him2 = resize(cv2.imread('./images/image_mosaicing/img5_2.jpg'), 40)
him3 = resize(cv2.imread('./images/image_mosaicing/img5_3.jpg'), 40)
him4 = resize(cv2.imread('./images/image_mosaicing/img5_4.jpg'), 40)

In [14]:
stitch_pano(him1, him3, 'him13')
him_temp_1 = resize(cv2.imread('./Results/uncrop_pano_him13.jpg'), 40)
stitch_pano(him_temp_1, him2, 'him132')
him_temp_2 = resize(cv2.imread('./Results/uncrop_pano_him132.jpg'), 40)
stitch_pano(him_temp_2, him4, 'him')

No. of matches obtained =  (7210, 2)
Homography Matrix:
[[ 1.46336981e+00  1.21535145e-01 -6.96334131e+02]
 [ 6.91326360e-02  1.37126729e+00 -6.63710655e+02]
 [ 1.79062737e-04  2.11533310e-04  1.00000000e+00]]
No. of matches obtained =  (3933, 2)
Homography Matrix:
[[ 2.22365120e+00 -2.97467051e-01 -5.34705248e+02]
 [-1.99367500e-02  1.99736654e+00 -1.36564799e+02]
 [ 1.07259314e-05 -3.56937482e-04  1.00000000e+00]]
No. of matches obtained =  (4243, 2)
Homography Matrix:
[[ 1.56981163e+00  5.85740058e-01 -1.57655495e+02]
 [-2.49983003e-01  2.14304581e+00 -3.62734488e+02]
 [-4.32920256e-04  3.43929103e-04  1.00000000e+00]]


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [16]:
tm1 = cv2.imread('./images/image_mosaicing/img2_1.png')
tm2 = cv2.imread('./images/image_mosaicing/img2_2.png')
tm3 = cv2.imread('./images/image_mosaicing/img2_3.png')
tm4 = cv2.imread('./images/image_mosaicing/img2_4.png')
tm5 = cv2.imread('./images/image_mosaicing/img2_5.png')
tm6 = cv2.imread('./images/image_mosaicing/img2_6.png')

In [17]:
tmf = stitch_pano(stitch_pano(stitch_pano(tm4,stitch_pano(stitch_pano(tm1,tm2,idx = 'tm_temp12',option =2),tm3,idx = 'tm_temp123',option =2),idx = 'tm_temp4123',option =2),tm5,idx = 'tm_temp41235',option =2),tm6,idx = 'tm',option =2)

No. of matches obtained =  (1486, 2)
Homography Matrix:
[[ 9.99941080e-01 -4.11363788e-05 -1.82984265e+02]
 [ 2.53608849e-05  9.99840331e-01 -9.98903910e+00]
 [-5.16090160e-08 -1.99081949e-07  1.00000000e+00]]
No. of matches obtained =  (1851, 2)
Homography Matrix:
[[ 9.99912560e-01 -7.11784232e-05 -4.74948114e+02]
 [-2.80470480e-05  9.99889004e-01  2.01577202e+00]
 [-1.12005610e-07 -1.08132909e-07  1.00000000e+00]]
No. of matches obtained =  (906, 2)
Homography Matrix:
[[ 9.99507867e-01 -4.22715444e-04 -4.92067519e+00]
 [ 2.15426866e-05  9.98893271e-01  1.39044213e+02]
 [-3.22899526e-07 -2.24681395e-06  1.00000000e+00]]
No. of matches obtained =  (2372, 2)
Homography Matrix:
[[ 1.00005632e+00  1.29784643e-04 -1.01042327e+02]
 [-2.65147963e-05  9.99836977e-01 -2.20935942e+02]
 [ 1.50775295e-08  1.64230798e-07  1.00000000e+00]]
No. of matches obtained =  (3228, 2)
Homography Matrix:
[[ 1.00029210e+00  9.77813295e-05 -4.26155505e+02]
 [-1.00419651e-05  1.00029507e+00 -2.45065451e+02]
 [ 

In [20]:
room1 = cv2.imread('./images/image_mosaicing/im1.jpg')
room2 = cv2.imread('./images/image_mosaicing/im2.jpg')
room3 = cv2.imread('./images/image_mosaicing/im3.jpg')
room4 = cv2.imread('./images/image_mosaicing/im4.jpg')

In [22]:
room_pano = stitch_pano(stitch_pano(stitch_pano(room1,room2,idx = 'room12',option = 2),room3, idx = 'room123',option = 2),room4,idx = 'room',option = 2)

No. of matches obtained =  (18225, 2)
Homography Matrix:
[[ 1.25600637e+00 -1.29018181e-01 -1.72791209e+03]
 [-2.25727332e-02  1.13118090e+00  9.89540862e+01]
 [ 7.95126048e-05 -6.88014735e-05  1.00000000e+00]]
No. of matches obtained =  (21160, 2)
Homography Matrix:
[[-3.94255643e-01  3.54495040e+00  1.22889973e+03]
 [-7.43799130e-01  1.58473823e+01  1.37615621e+03]
 [-4.33918570e-04  7.46981234e-03  1.00000000e+00]]
No. of matches obtained =  (11006, 2)
Homography Matrix:
[[ 1.43423647e+00  5.16673581e-02 -1.90091006e+03]
 [ 4.40426729e-02  1.29317179e+00 -1.33290775e+02]
 [ 9.14301773e-05  4.07347504e-06  1.00000000e+00]]


### Panorama Stitching using Homography from scratch

In [24]:
def homography(pts1,pts2):
    '''
    A function to compute the homography matrix based on previous work in DLT
    '''
    pt1 = np.insert(pts1,2,1,axis=1)
    wp = np.stack((pt1,pt1),2).transpose(0,2,1).reshape(2*pt1.shape[0],3)
    wp_mat = np.hstack((wp,wp,wp))
    tmp_mat = np.tile(np.hstack((np.vstack((-1*np.ones(3),np.zeros(3))),np.vstack((np.zeros(3),-1*np.ones(3))))),(pts2.shape[0],1))
    im_mat = np.hstack((tmp_mat,np.tile(pts2.ravel()[:,None],(1,3))))
    A = im_mat*wp_mat
    
    U,D,V = np.linalg.svd(A)
    h = V[-1,:]
    H = h.reshape(3,3)
    return H/H[2,2]

def error(pts1,pts2,M):
    '''
    A function to compute reprojection error.
    '''
    proj_pts = M.dot(np.insert(pts1,2,1,axis=1).T)
    proj_pts = (proj_pts/proj_pts[-1,:])[:2,:].T
    return np.sum(np.linalg.norm(proj_pts-pts2,axis=1))/pts1.shape[0]

def ransac_homography(pts1,pts2,sample_sz = 30,max_iter=1000):
    '''
    A function to compute the homography matrix and optimises the matrix based on running RANSAC for a 
    smaller set of points.
    '''
    errr = 1e6
    H0 = np.empty((3,3))
    for i in range(max_iter):
        ind = np.random.choice(pts1.shape[0],sample_sz,replace = False)
        pt1 = pts1[ind,:]
        pt2 = pts2[ind,:]
        H = homography(pt1,pt2)
        err = error(pt1,pt2,H)
        if(err<errr):
            errr = err
            H0 = H
    return H0

In [25]:
def stitch_pano_self(image1, image2, idx, option = 1, factor = 2):
    '''
    Stitch two input images based on RANSAC based homography matrix.
    '''
    keypointsImage1, descriptorImage1 = applySIFT(image1)
    keypointsImage2, descriptorImage2 = applySIFT(image2)

    Image1Keypoints=cv2.drawKeypoints(image1,keypointsImage1,None)
    cv2.imwrite('Results/self/sift1_'+str(idx)+'.jpg',Image1Keypoints)
    cv2.imshow('keypoints in Image 1',Image1Keypoints)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    Image2Keypoints=cv2.drawKeypoints(image2,keypointsImage2,None)
    cv2.imwrite('Results/self/sift2_'+str(idx)+'.jpg',Image2Keypoints)
    cv2.imshow('keypoints in Image 2',Image2Keypoints)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    goodList, good = getGoodMatches(descriptorImage1, descriptorImage2, factor=factor)

    imagePlot = cv2.drawMatchesKnn(image1,keypointsImage1,image2,keypointsImage2,goodList,None,flags=2)
    cv2.imwrite('Results/self/matches_knn_'+str(idx)+'.jpg',imagePlot)
    cv2.imshow('matches derived by 2nn',imagePlot)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    ptsImage1 = np.array([ keypointsImage1[m.queryIdx].pt for m in good]).reshape(-1,2)
    ptsImage2 = np.array([ keypointsImage2[m.trainIdx].pt for m in good]).reshape(-1,2)

    H = ransac_homography(ptsImage1, ptsImage2)
    print('Homography Matrix:')
    print(H)
    
    if (option == 1):
        h, w, d = image1.shape
        image1Corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        image1CornersPlane2 = np.squeeze(cv2.perspectiveTransform(image1Corners,H))
        cv2.destroyAllWindows()
    
        xMin, yMin, xMax, yMax = getExtremePoints(image1CornersPlane2)
        t1 = (xMax-xMin, yMax-yMin)
        t2 = (len(image2[0])-int(xMin), len(image2)-int(yMin))
        finalImageShape = max(t1,t2)
        if xMin < 0 and yMin < 0:
            translate = np.float32([[1,0, -xMin], [0,1, -yMin], [0,0,1]])
        elif xMin < 0:
            translate = np.float32([[1,0, -xMin], [0,1,0], [0,0,1]])
        elif xMin < 0:
            translate = np.float32([[1,0,0], [0,1, -yMin], [0,0,1]])
        else:
            translate = np.float32([[1,0,0], [0,1,0], [0,0,1]])
        finalImage = cv2.warpPerspective(image1, np.matmul(translate,H), finalImageShape)
        finalImage[-int(yMin):-int(yMin)+image2.shape[0], -int(xMin):-int(xMin)+image2.shape[1]]=image2
    
        cv2.imwrite('Results/self/uncrop_pano_'+str(idx)+'.jpg',finalImage)
        cv2.imshow('Uncropped Pano',finalImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
        return finalImage
    
    elif (option == 2):
        h1, w1 = image2.shape[:2]
        h2, w2 = image1.shape[:2]
        c1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
        c2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
        c2_ = cv2.perspectiveTransform(c2, H)
        c = np.concatenate((c1, c2_), axis=0)

        [xmin, ymin] = np.int32(c.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(c.max(axis=0).ravel() + 0.5)
        t = [-xmin, -ymin]

        Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])

        out = cv2.warpPerspective(image1, Ht.dot(H), (xmax-xmin, ymax-ymin))
        out[t[1]:h1+t[1], t[0]:w1+t[0]] = image2
        finalImage = out
        cv2.imwrite('Results/self/uncrop_pano_'+str(idx)+'.jpg',finalImage)
        cv2.imshow('Uncropped Pano',finalImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        return finalImage


In [26]:
stitch_pano_self(scottsdale1, scottsdale2, 'scottsdale', option = 1, factor=0.75)

No. of matches obtained =  (2240, 2)
Homography Matrix:
[[ 2.49056273e+00 -1.14195603e-01 -1.36320001e+03]
 [ 5.54331296e-01  2.12058950e+00 -4.70892586e+02]
 [ 1.48498666e-03 -1.06105544e-05  1.00000000e+00]]


array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [56, 65, 87],
        [54, 66, 84],
        [52, 66, 84]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [52, 63, 84],
        [51, 64, 82],
        [51, 65, 83]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [50, 61, 82],
        [52, 66, 84],
        [54, 69, 86]]], dtype=uint8)

In [23]:
stitch_pano_self(amphi1, amphi2, 'amphi12',option=1, factor=0.75)
amphi_temp_1 = resize(cv2.imread('./Results/self/uncrop_pano_amphi12.jpg'), 40)
stitch_pano_self(amphi_temp_1, amphi3, 'amphi123',option=1, factor=0.75)

No. of matches obtained =  (9587, 2)
Homography Matrix:
[[ 1.72275271e+00  4.02524702e-03 -1.04179886e+03]
 [ 1.76598323e-01  1.47115271e+00 -2.34243686e+02]
 [ 3.90719391e-04  2.44463586e-05  1.00000000e+00]]
No. of matches obtained =  (4306, 2)
Homography Matrix:
[[ 4.65024490e+00 -7.18000276e-02 -2.72644227e+03]
 [ 3.41386798e-01  4.07657872e+00 -6.76343109e+02]
 [ 7.69163812e-04 -5.12789168e-05  1.00000000e+00]]


array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [41, 50, 54],
        [40, 49, 53],
        [39, 48, 51]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [40, 49, 52],
        [39, 48, 52],
        [41, 50, 53]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [41, 50, 53],
        [40, 49, 52],
        [33, 42, 45]]], dtype=uint8)

In [24]:
amphi_temp_2 = resize(cv2.imread('./Results/self/uncrop_pano_amphi123.jpg'), 40)
stitch_pano_self(amphi_temp_2, amphi4, 'amphi', option=1, factor=0.75)

No. of matches obtained =  (8029, 2)
Homography Matrix:
[[ 4.34737937e+00 -3.61502096e-02 -5.37702096e+03]
 [ 2.33436516e-01  3.99969650e+00 -1.43749789e+03]
 [ 5.15043612e-04  1.41828482e-05  1.00000000e+00]]


array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [137, 143, 151],
        [136, 143, 151],
        [133, 139, 147]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [135, 142, 151],
        [135, 142, 151],
        [133, 140, 149]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [136, 144, 154],
        [137, 145, 155],
        [132, 141, 151]]

In [25]:
stitch_pano_self(him1, him3, 'him13',option=1, factor=0.75)
him_temp_1 = resize(cv2.imread('./Results/uncrop_pano_him13.jpg'), 40)
stitch_pano_self(him_temp_1, him2, 'him132',option=1, factor=0.75)
him_temp_2 = resize(cv2.imread('./Results/uncrop_pano_him132.jpg'), 40)
stitch_pano_self(him_temp_2, him4, 'him',option=1, factor=0.75)

No. of matches obtained =  (7210, 2)
Homography Matrix:
[[ 1.45199464e+00  1.18788786e-01 -6.89276980e+02]
 [ 6.83203826e-02  1.36000348e+00 -6.57569208e+02]
 [ 1.75361492e-04  2.03571997e-04  1.00000000e+00]]
No. of matches obtained =  (3933, 2)
Homography Matrix:
[[ 2.22710077e+00 -2.95705682e-01 -5.35819832e+02]
 [-1.95401701e-02  2.00252687e+00 -1.37576855e+02]
 [ 1.17749662e-05 -3.53990159e-04  1.00000000e+00]]
No. of matches obtained =  (4243, 2)
Homography Matrix:
[[ 1.55636675e+00  5.70095639e-01 -1.51410890e+02]
 [-2.49128542e-01  2.12507502e+00 -3.59076518e+02]
 [-4.34322564e-04  3.27666401e-04  1.00000000e+00]]


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [26]:
tm1 = cv2.imread('./images/image_mosaicing/img2_1.png')
tm2 = cv2.imread('./images/image_mosaicing/img2_2.png')
tm3 = cv2.imread('./images/image_mosaicing/img2_3.png')
tm4 = cv2.imread('./images/image_mosaicing/img2_4.png')
tm5 = cv2.imread('./images/image_mosaicing/img2_5.png')
tm6 = cv2.imread('./images/image_mosaicing/img2_6.png')

In [28]:
tmf = stitch_pano_self(stitch_pano_self(stitch_pano_self(tm4,stitch_pano_self(stitch_pano_self(tm1,tm2,idx = 'tm_temp12',option =2, factor = 0.75),tm3,idx = 'tm_temp123',option =2, factor = 0.75),idx = 'tm_temp4123',option =2, factor = 0.75),tm5,idx = 'tm_temp41235',option =2, factor = 0.75),tm6,idx = 'tm',option =2, factor = 0.75)

No. of matches obtained =  (1486, 2)
Homography Matrix:
[[ 1.00000000e+00 -3.92972031e-09 -1.83000000e+02]
 [ 5.28042998e-09  9.99999984e-01 -1.00000001e+01]
 [ 2.26865896e-11 -6.53728731e-11  1.00000000e+00]]
No. of matches obtained =  (1851, 2)
Homography Matrix:
[[ 1.00000034e+00  1.61176434e-08 -4.75000180e+02]
 [ 8.98146654e-08  1.00000025e+00  1.99994465e+00]
 [ 3.18350527e-10  2.05279801e-10  1.00000000e+00]]
No. of matches obtained =  (906, 2)
Homography Matrix:
[[ 9.99999921e-01 -2.15011020e-08 -4.99999425e+00]
 [-6.55185810e-08  9.99999965e-01  1.39000000e+02]
 [-1.71356966e-10 -1.65654825e-10  1.00000000e+00]]
No. of matches obtained =  (2361, 2)
Homography Matrix:
[[ 9.99999981e-01  3.16975033e-08 -1.01000003e+02]
 [-8.42010381e-09  1.00000007e+00 -2.21000019e+02]
 [-8.09050723e-11  1.58598356e-10  1.00000000e+00]]
No. of matches obtained =  (3225, 2)
Homography Matrix:
[[ 1.00000011e+00  1.55464464e-08 -4.26000055e+02]
 [ 9.40094707e-09  1.00000024e+00 -2.45000071e+02]
 [-

In [69]:
room1 = cv2.imread('./images/image_mosaicing/im1.jpg')
room2 = cv2.imread('./images/image_mosaicing/im2.jpg')
room3 = cv2.imread('./images/image_mosaicing/im3.jpg')
room4 = cv2.imread('./images/image_mosaicing/im4.jpg')

In [70]:
room_pano = stitch_pano_self(stitch_pano_self(stitch_pano_self(room1,room2,idx = 'room12',option = 2),room3, idx = 'room123',option = 2),room4,idx = 'room',option = 2)

No. of matches obtained =  (18225, 2)
Homography Matrix:
[[-5.28388839e-01 -2.05333653e-01  1.43443054e+03]
 [-2.77638916e-01 -6.22344602e-02  7.29653367e+02]
 [-3.39783591e-04 -2.41140134e-04  1.00000000e+00]]
No. of matches obtained =  (18915, 2)
Homography Matrix:
[[-4.43853499e-01 -1.50754472e-01  9.65102146e+02]
 [-7.56899334e-01 -3.23479157e-01  1.83908041e+03]
 [-4.00643478e-04 -1.83317943e-04  1.00000000e+00]]
No. of matches obtained =  (11006, 2)
Homography Matrix:
[[-6.07631764e-01 -7.02570392e-01  2.47604204e+03]
 [-5.16189966e-01 -5.40771653e-01  1.98812318e+03]
 [-2.47497400e-04 -2.76706145e-04  1.00000000e+00]]


*Think of an algorithm which can stitch images given in any order without human
intervention? If yes, modify your exisitng code accordingly.*

*Is it possible to stitch a panorama without human intervention, given noisy images
which do not belong to the same scene? If yes, how?*

## Task 2 
## Stereo Correspondences

In [12]:
def corr(v1,v2):
    '''
    A function that computes correlation between pixels.
    '''
    return v1.T.dot(v2)/(np.sqrt(v1.T.dot(v1))*np.sqrt(v2.T.dot(v2)))

def correlation_matching(img1,img2,window_size=45,stride=45):
    '''
    A function that computes correlation between pixels and stores the pixels with best matches.
    '''
    h1,w1,c = img1.shape
    h2,w2,c = img2.shape
    
#     Pass through all the patches in img1 and find patch in img2 with least 
    best_matches = []
    for y1 in range(0,h1-window_size,stride):
        for x1 in range(0,w1-window_size,stride):
            least_dis = 4.0
            for y2 in range(0,h2-window_size,stride):
                for x2 in range(0,w2-window_size,stride):
                    v1 = img1[y1:y1+window_size, x1:x1+window_size,:].flatten()
                    v2 = img2[y2:y2+window_size, x2:x2+window_size,:].flatten()
                    dis = corr(v1,v2)
                    if least_dis > dis:
                        least_dis = dis
                        least_coord = [x1,y1,x2,y2,dis]
            best_matches.append(least_coord)
    return best_matches

def draw_matches(img,matches,window_size=128):
    '''
    A helper function to draw the matches stored in above function.
    '''
    h,w,c = img.shape
    for match in matches:
        pt1 = (match[1]+window_size//2,match[0]+window_size//2)
        pt2 = (match[3]+window_size//2+w//2,match[2]+ window_size//2)
        line_img = cv2.line(img,pt1,pt2,(0,0,225),3)
    
    return line_img

In [39]:
img = cv2.imread('./images/stereo_images/1.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
matches = correlation_matching(img1,img2)
dis_im = draw_matches(img,matches)
cv2.imwrite('./Results/q2/Dense_Pair1.jpg',dis_im)
cv2.imshow('Dense Pair',dis_im)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Case 2
img = cv2.imread('./images/stereo_images/2.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
matches = correlation_matching(img1,img2)
dis_im = draw_matches(img,matches)
cv2.imwrite('./Results/q2/Dense_Pair2.jpg',dis_im)
cv2.imshow('Dense Pair',dis_im)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Case 3
img = cv2.imread('./images/stereo_images/3.jpeg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
matches = correlation_matching(img1,img2)
dis_im = draw_matches(img,matches)
cv2.imwrite('./Results/q2/Dense_Pair3.jpg',dis_im)
cv2.imshow('Dense Pair',dis_im)
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/soumyasis/.virtualenvs/cvit1/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


### Using Dynamic time Warping to find matches

In [38]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' 
    A function to draw the epilines for the points in image 2 on image 1 
    where input lines are the corresponding epilines
    '''
    r,c,_ = img1.shape
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

def dp_(line1, line2):
    dp = np.zeros((line1.shape[0]+1,line2.shape[0]+1))
    
    for i,x in enumerate(line1):
        for j,y in enumerate(line2):
            if(i==0 or j==0):
                continue
            elif (line1[i-1] == line1[j-1]): 
                dp[i,j] = 1 + dp[i-1][j-1]
            else:
                dp[i,j] = max(dp[i-1,j], dp[i,j-1])

    i = line1.shape[0], j = line2.shape[0]; 
    newline = np.zeros(line1.shape[0])
    index = line1.shape[0]
    while (i > 0 and j > 0) : 
        if (line1[i-1] == line2[j-1]): 
            newline[index-1] = line1[i-1]
            i-=1
            j-=1
            index-=1 
      
        elif (dp[i-1,j] > dp[i,j-1]): 
            i-=1 
        else:
            j-=1
    return newline

def plot_epipolar_lines(img1,img2,sift_params,idx,match_th =0.8):
    '''
    A function to draw the epipolar lines with the best matches on those lines. 
    '''
    kp1,des1,kp2,des2,_,_ = sift_params
    index_params = dict(algorithm = 1, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    for i,(m,n) in enumerate(matches):
        if m.distance < match_th*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.RANSAC)
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img3,img4 = drawlines(img1,img2,lines1,pts1,pts2)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img5,img6 = drawlines(img2,img1,lines2,pts2,pts1)
    
    cv2.imwrite('./Results/q2/Parallel epilines img1_'+str(idx)+'.jpg',img5)
    cv2.imwrite('./Results/q2/Parallel epilines img2_'+str(idx)+'.jpg',img6)
    cv2.imshow('Epiline img 1',img5)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imshow('Epiline img 2',img6)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return lines1,lines2

In [53]:
img = cv2.imread('./images/stereo_images/1.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2,idx = 1)
l1,l2 = plot_epipolar_lines(img1,img2,sift_param, idx = 1)

In [54]:
img = cv2.imread('./images/stereo_images/2.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2,idx = 2)
l1, l2 = plot_epipolar_lines(img1,img2,sift_param,idx = 2)

In [55]:
img = cv2.imread('./images/stereo_images/3.jpeg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2, idx = 3)
l1,l2 = plot_epipolar_lines(img1,img2,sift_param, idx =3)

### Drawing Epilines on stereo rectified images

In [56]:
def dense_sift_matching(img1,img2,idx,min_match_cnt=500):
    '''
    A function to derive sift features and run a brute force match to find correspondences
    '''
    cv2.imshow('Image 1',img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imshow('Image 2',img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    sift = cv2.xfeatures2d.SIFT_create()
    h,w,c = img1.shape
    kp=[]
    for i in range(1,h,10):
        for j in range(1,w,10):
            kp.append(cv2.KeyPoint(i, j, 3))
    
    gray_im1 = cv2.cvtColor(img1,cv2.COLOR_RGB2GRAY)
    kp1,des1 = sift.compute(gray_im1,kp)

    gray_im2 = cv2.cvtColor(img2,cv2.COLOR_RGB2GRAY)
    kp2,des2 = sift.compute(gray_im2,kp)

    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)[0:min_match_cnt]
    draw_params = dict(matchesMask=None,singlePointColor=None,flags=2)
        
    dis_im = cv2.drawMatches(img1,kp1,img1,kp1,matches,None,**draw_params)
    cv2.imshow('Matches',dis_im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return kp1,des1,kp2,des2,matches,dis_im

In [57]:
def stereo_rectification(img1,img2,sift_params,match_th =0.8):
    '''
    A function that uses sift features to rectify two input images
    i.e. make the epilines parallel in both images to greedily or using dp verify the
    match.
    '''
    kp1,des1,kp2,des2,_,_ = sift_params
    
    index_params = dict(algorithm = 1, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    for i,(m,n) in enumerate(matches):
        if m.distance < match_th*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.RANSAC)
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    img_size = img1.shape[0:2]
    p,H1,H2=cv2.stereoRectifyUncalibrated(pts1, pts2, F, img_size)
        
    H3= H1.dot(H2)
    img1_corrected = cv2.warpPerspective(img1, H1, img_size)
    img2_corrected = cv2.warpPerspective(img2, H3, img_size)
    
    return img1_corrected, img2_corrected


In [64]:
img = cv2.imread('./images/stereo_images/1.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2,idx = 1)
rect_img1, rect_img2 = stereo_rectification(img1,img2,sift_param,match_th =0.8)
cv2.imwrite('./Results/q2/Stereo_Rect1_1.jpg',rect_img1)
cv2.imshow('Stereo Rectified image 1',rect_img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./Results/q2/Stereo_Rect1_2.jpg',rect_img2)
cv2.imshow('Stereo Rectified image 2',rect_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
img = cv2.imread('./images/stereo_images/2.jpg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2, idx = 2)
rect_img1, rect_img2 = stereo_rectification(img1,img2,sift_param,match_th =0.8)
cv2.imwrite('./Results/q2/Stereo_Rect2_1.jpg',rect_img1)
cv2.imshow('Stereo Rectified image 1',rect_img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./Results/q2/Stereo_Rect2_2.jpg',rect_img2)
cv2.imshow('Stereo Rectified image 2',rect_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
img = cv2.imread('./images/stereo_images/3.jpeg')
h,w,c = img.shape
img1 = img[:,0:w//2,:]
img2 = img[:,w//2:w,:]
sift_param = dense_sift_matching(img1,img2, idx = 3)
rect_img1, rect_img2 = stereo_rectification(img1,img2,sift_param,match_th =1.1)
cv2.imwrite('./Results/q2/Stereo_Rect3_1.jpg',rect_img1)
cv2.imshow('Stereo Rectified image 1',rect_img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('./Results/q2/Stereo_Rect3_2.jpg',rect_img2)
cv2.imshow('Stereo Rectified image 2',rect_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
img1 = cv2.imread('./Results/q2/Stereo_Rect1_1.jpg')
img2 = cv2.imread('./Results/q2/Stereo_Rect1_2.jpg')
sift_param = dense_sift_matching(img1,img2,idx = 'rect1')
l1,l2 = plot_epipolar_lines(img1,img2,sift_param, idx = 'rect1')

In [66]:
img1 = cv2.imread('./Results/q2/Stereo_Rect2_1.jpg')
img2 = cv2.imread('./Results/q2/Stereo_Rect2_2.jpg')
sift_param = dense_sift_matching(img1,img2,idx = 'rect2')
l1,l2 = plot_epipolar_lines(img1,img2,sift_param, idx = 'rect2')

In [68]:
img1 = cv2.imread('./Results/q2/Stereo_Rect3_1.jpg')
img2 = cv2.imread('./Results/q2/Stereo_Rect3_2.jpg')
sift_param = dense_sift_matching(img1,img2,idx = 'rect3')
l1,l2 = plot_epipolar_lines(img1,img2,sift_param, idx = 'rect3')